In [1]:
import tensorflow as tf
import os
import numpy as np
import random
from tensorflow.keras.applications import resnet
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

##from pyexcel.cookbook import merge_all_to_a_book
# import pyexcel.ext.xlsx # no longer required if you use pyexcel >= 0.2.2 
import glob

def get_embedding_module(imageSize):
    # construct the input layer and pass the inputs through a
    # pre-processing layer
    inputs = keras.Input(imageSize + (3,))
    x = resnet.preprocess_input(inputs)

    # fetch the pre-trained resnet 50 model and freeze the weights
    baseCnn = resnet.ResNet50(weights="imagenet", include_top=False)
    baseCnn.trainable=False

    # pass the pre-processed inputs through the base cnn and get the
    # extracted features from the inputs
    extractedFeatures = baseCnn(x)
    # pass the extracted features through a number of trainable layers
    x = layers.GlobalAveragePooling2D()(extractedFeatures)
    x = layers.Dense(units=1024, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(units=128)(x)
    # build the embedding model and return it
    embedding = keras.Model(inputs, outputs, name="embedding")
    return embedding

def get_siamese_network(imageSize, embeddingModel):
    # build the anchor, positive and negative input layer
    anchorInput = keras.Input(name="anchor", shape=imageSize + (3,))
    positiveInput = keras.Input(name="positive", shape=imageSize + (3,))
    negativeInput = keras.Input(name="negative", shape=imageSize + (3,))
    # embed the anchor, positive and negative images
    anchorEmbedding = embeddingModel(anchorInput)
    positiveEmbedding = embeddingModel(positiveInput)
    negativeEmbedding = embeddingModel(negativeInput)
    # build the siamese network and return it
    siamese_network = keras.Model(
        inputs=[anchorInput, positiveInput, negativeInput],
        outputs=[anchorEmbedding, positiveEmbedding, negativeEmbedding]
    )
    return siamese_network

class SiameseModel(keras.Model):
    def __init__(self, siameseNetwork, margin, lossTracker):
        super().__init__()
        self.siameseNetwork = siameseNetwork
        self.margin = margin
        self.lossTracker = lossTracker
    def _compute_distance(self, inputs):
        (anchor, positive, negative) = inputs
        # embed the images using the siamese network
        embeddings = self.siameseNetwork((anchor, positive, negative))
        anchorEmbedding = embeddings[0]
        positiveEmbedding = embeddings[1]
        negativeEmbedding = embeddings[2]
        # calculate the anchor to positive and negative distance
        apDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - positiveEmbedding), axis=-1
        )
        anDistance = tf.reduce_sum(
            tf.square(anchorEmbedding - negativeEmbedding), axis=-1
        )

        # return the distances
        return (apDistance, anDistance)
    def _compute_loss(self, apDistance, anDistance):
        loss = apDistance - anDistance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss
    def call(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        (apDistance, anDistance) = self._compute_distance(inputs)
        return (apDistance, anDistance)
    def train_step(self, inputs):
        with tf.GradientTape() as tape:
            # compute the distance between the anchor and positive,
            # negative images
            (apDistance, anDistance) = self._compute_distance(inputs)
            # calculate the loss of the siamese network
            loss = self._compute_loss(apDistance, anDistance)
        # compute the gradients and optimize the model
        gradients = tape.gradient(
            loss,
            self.siameseNetwork.trainable_variables)
        self.optimizer.apply_gradients(
            zip(gradients, self.siameseNetwork.trainable_variables)
        )
        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}
    def test_step(self, inputs):
        # compute the distance between the anchor and positive,
        # negative images
        (apDistance, anDistance) = self._compute_distance(inputs)
        # calculate the loss of the siamese network
        loss = self._compute_loss(apDistance, anDistance)

        # update the metrics and return the loss
        self.lossTracker.update_state(loss)
        return {"loss": self.lossTracker.result()}
    @property
    def metrics(self):
        return [self.lossTracker]


################################################3




2024-10-16 08:11:05.688773: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 08:11:05.753492: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-16 08:11:06.090368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 08:11:06.090515: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 08:11:06.138336: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
# sanour and salah

import cv2
import numpy as np
import os
from pathlib import Path
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from time import sleep
from imutils.video import VideoStream
import imutils

IMAGES_PATH = '/home/salah/Documents/Attendence-Project-main/zdraw_augmented_copy'
MODEL_PATH = 'siamese_network_black_and_white'
H5_MODEL_PATH = "./mymodel.h5"
# CASCADE_PATH = 'haarcascade_frontalface_default.xml'
THRESHOLD = 0.05

# Show waiting screen
waiting = np.zeros(500, np.uint8)
cv2.imshow("WebCam", waiting)

# Load the model
modelPath = MODEL_PATH
print(f"[INFO] loading the siamese network from {modelPath}...")
siameseNetwork = keras.models.load_model(filepath=H5_MODEL_PATH)


siameseModel = SiameseModel(
	siameseNetwork=siameseNetwork,
	margin=0.5,
	lossTracker=keras.metrics.Mean(name="loss"),
)


def to_gray(images):
    gray_imgs = []
    for img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray_imgs.append(gray)
  
    return gray_imgs


def find_embeddings(images):
    """
    A function to find each face embeddings for the faces in each image
    in @images

    Args:
        - images: a list of images
    """
    images_embeddings = []
    for image in images:
        gray_faces = to_gray([image])
        for i, gray_face in enumerate(gray_faces):
            cv2.imwrite(f'temp/{i}.jpg', gray_face)

            gray_face = tf.io.read_file(f'temp/{i}.jpg')
            gray_face = tf.image.decode_image(gray_face, channels=3)
            gray_face = tf.image.resize(gray_face, (125,125))
            gray_face = tf.cast(gray_face, tf.float32) / 255.0
            gray_face = tf.expand_dims(gray_face, axis=0)

            embeddings = siameseModel.siameseNetwork((gray_face, gray_face, gray_face))[0]
            images_embeddings.append(embeddings)
    return images_embeddings, images_embeddings


      





[INFO] loading the siamese network from siamese_network_black_and_white...


In [4]:
# Caching --> calculate the images embeddings
#######################################
# A list to save embeddings
Path = IMAGES_PATH
images = []
ClassNames = []
MyList = os.listdir(Path)
print(MyList)
for cl in MyList:
    curImg = cv2.imread(f'{Path}/{cl}')
    images.append(curImg)
    ClassNames.append(os.path.splitext(cl)[0])
print(ClassNames)



students_embedding, _ =  find_embeddings(images)
# now we already have the images stored in 
print(students_embedding)

['123159.png', '123468.png', '123251.png', '123115.png', '123276.png', '123070.png', '123543.png', '123211.png', '123130.png', '123233.png', '123023.png', '123155.png', '123321.png', '123391.png', '123193.png', '123144.png', '123357.png', '123272.png', '123488.png', '123006.png', '123279.png', '123277.png', '123099.png', '123076.png', '123545.png', '123275.png', '123425.png', '123406.png', '123074.png', '123485.png', '123093.png', '123064.png', '123378.png', '123247.png', '123165.png', '123499.png', '123198.png', '123234.png', '123455.png', '123395.png', '123367.png', '123184.png', '123544.png', '123139.png', '123094.png', '123181.png', '123483.png', '123469.png', '123137.png', '123501.png', '123458.png', '123207.png', '123183.png', '123497.png', '2.png', '123141.png', '123035.png', '123409.png', '1241421.png', '123127.png', '123046.png', '123297.png', '123256.png', '123496.png', '1234124.png', '123222.png', '123355.png', '123238.png', '123227.png', '123135.png', '123331.png', '123154.

In [93]:
################################################ #

################################################# #

path1 = '/home/salah/Documents/Attendence-Project-main/tst.png'
path2 = '/home/salah/Documents/Attendence-Project-main/image.png'
path3 = '/home/salah/Documents/Attendence-Project-main/11221122.png'
path4 = '/home/salah/Documents/Attendence-Project-main/fl.png'

embeddings_draw = find_embeddings([cv2.imread(path4)])
lst = []
cnt = 0
for i, student_embedding in enumerate(students_embedding):
    cnt += 1
    distance = tf.reduce_sum(tf.square(student_embedding - embeddings_draw), axis=-1)
    distance = float(distance[0])
    print(f"the dist is {distance}")
    lst.append(distance)

threshold = 1.0  
cntr = 0

for item in lst:
    if item <= threshold:
        cntr += 1

if cntr >= 100:
    print("Matching detected!")
else:
    print("No matching detected.")

print(f"Number of embeddings with distance <= {threshold}: {cntr}")
print(f"Total number of embeddings checked: {cnt}")


the dist is 0.07268255203962326
the dist is 0.39628279209136963
the dist is 0.2940220832824707
the dist is 0.1341317892074585
the dist is 0.30429935455322266
the dist is 0.0895187258720398
the dist is 0.11762682348489761
the dist is 0.23056960105895996
the dist is 0.3885476887226105
the dist is 0.2189089059829712
the dist is 0.40024375915527344
the dist is 0.17529156804084778
the dist is 0.15191572904586792
the dist is 0.2025739550590515
the dist is 0.16090437769889832
the dist is 0.047916658222675323
the dist is 0.04105686396360397
the dist is 0.3162047863006592
the dist is 0.1784433275461197
the dist is 0.5309576392173767
the dist is 0.07395395636558533
the dist is 0.31563353538513184
the dist is 0.4122934341430664
the dist is 1.1692845821380615
the dist is 0.25916168093681335
the dist is 0.07112184166908264
the dist is 0.12394029647111893
the dist is 0.9744699597358704
the dist is 1.27488112449646
the dist is 0.32330322265625
the dist is 0.3440302610397339
the dist is 0.190685123205

In [69]:

embeddings_draw = find_embeddings([cv2.imread(path1)])[0]  

lst = []
cnt = 0
for student_embedding in  enumerate(students_embedding):
    cnt += 1
    # Using cosine similarity instead of squared difference
    student_embedding = tf.convert_to_tensor(student_embedding, dtype=tf.float32)
    similarity = tf.keras.losses.cosine_similarity(student_embedding, embeddings_draw)
    similarity = float(similarity)
    print(f"Cosine similarity: {similarity}")
    lst.append(similarity)

# # Convert the list of similarities to numpy array
# lst = np.array(lst)

# # Using dynamic thresholds based on the distribution of similarities
# mean_similarity = np.mean(lst)
# std_similarity = np.std(lst)

# # Set a dynamic threshold (e.g., mean - 1.5 * standard deviation)
# dynamic_threshold = mean_similarity - 1.5 * std_similarity
# print(f"Dynamic Threshold for similarity: {dynamic_threshold}")

# # Soft voting mechanism
# similarity_scores = lst[lst >= dynamic_threshold]  # Count scores above the dynamic threshold
# soft_vote_count = len(similarity_scores)

# # Calculate the soft voting similarity ratio
# similarity_ratio = soft_vote_count / cnt if cnt > 0 else 0
# print(f"Number of embeddings above dynamic threshold: {soft_vote_count}")
# print(f"Total embeddings: {cnt}")
# print(f"Similarity Ratio: {similarity_ratio:.2f}")

# # Decision based on similarity ratio
# similarity_threshold = 0.6  # This threshold can be adjusted
# if similarity_ratio >= similarity_threshold:
#     print("Big similarity detected!")
# else:
#     print("No significant similarity detected.")

# # Additional metrics
# print("Minimum similarity (cosine):", min(lst))
# print("Average similarity (cosine):", np.mean(lst))

InvalidArgumentError: {{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [] != values[1].shape = [1,128] [Op:Pack] name: 